In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.integrate import odeint
from __future__ import division
import time
import matplotlib.animation as animation
from IPython.display import clear_output, display
from IPython.html.widgets import FloatProgress

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [3]:
def equilibrate(fa, fb, fp, ka, kb):
    ka, kb = 1.0/ka, 1.0/kb
    a = ka + kb + fa + fb - fp
    b = kb*(fa-fp) + ka*(fb-fp) + ka*kb
    c = -ka*kb*fp
    
    theta = arccos((-2.0*a*a*a+9.0*a*b-27.0*c)/(2*sqrt((a*a-3.0*b)**3.0)))
    
    p = -(a/3.0) + (2.0/3.0)*sqrt(a*a-3.0*b)*cos(theta/3.0)
    pa = (fa*(2*sqrt(a*a-3*b)*cos(theta/3.0)-a)) / (3.0*ka+(2*sqrt(a*a-3*b)*cos(theta/3.0)-a))
    pb = (fb*(2*sqrt(a*a-3*b)*cos(theta/3.0)-a)) / (3.0*kb+(2*sqrt(a*a-3*b)*cos(theta/3.0)-a))
    
    a = fa - pa
    b = fb - pb
    
    fa / (a+pa)
    return a, b, p, pa, pb
    #return a*fa / (a+pa), b*fb/(b+pb), p*fp/(pa+pb+p), pa*fp / (pb+pa+b), pb*fp / (pb+pa+b)

species = 5
# protein, capture, analyte, control
prot_i, cap_i, anal_i, prot_cap_i, prot_anal_i = range(0,species)

def run_column(protein_start=10**-10.0, anal_start=10**-9.0, cap_start=10**-5.5, kanal=10**10.0, kcap=10**8.0, timestep=0.1, \
               verbose=False, dtype=longdouble, stop=2.0):
    width, length, thickness = 10.0, 50.0, 0.15#units of mm
    porosity = 0.7
    col_vol = porosity*width*length*thickness / 1000.0 #ml
    if verbose: print 'Column volume is %0.2f ul'%(col_vol*1000)

    flow = 0.01 #ml/min
    flow = flow/60.0 #ml/s
    plates = 100
    plate_vol = col_vol / plates
    plate_flow = flow/plate_vol
    if verbose: print 'Plate volume is %0.2f ul'%(plate_vol*1000)

    vstart, vstop = 0, col_vol*stop
    iterations = int(floor(vstop/(timestep*flow)))
    if verbose: print iterations
    volumes = array(range(0, iterations))*flow*timestep

    if verbose: print 'pf timestep product:',plate_flow*timestep

    col = zeros((plates, species), dtype=dtype)
    col[29:33, prot_i] = protein_start
    col[0:33, anal_i] = anal_start
    col[48, cap_i] = cap_start

    start = time.time()

    out = zeros((plates, species, iterations))
    for i in range(0, iterations):
        out[:,:,i] = col
        current_volume = i*timestep*flow

        prot = col[:,prot_i]
        cap = col[:,cap_i]
        anal = col[:,anal_i]
        prot_cap = col[:,prot_cap_i]
        prot_anal = col[:,prot_anal_i]

        col[:,anal_i], col[:,cap_i], col[:,prot_i], col[:,prot_anal_i], col[:,prot_cap_i] = \
            equilibrate(anal+prot_anal, cap+prot_cap, prot+prot_cap+prot_anal, kanal, kcap)
        col[isnan(col)] = 0

        for key in [prot_i, anal_i, prot_anal_i]:
            change = col[:,key].copy()*plate_flow*timestep
            if key != anal_i: change[58:]=0
            col[:,key] = col[:,key] - change[:]
            col[1:, key] = col[1:, key] + change[:-1]
            if key == anal_i: col[0, key] = col[0, key] + anal_start*plate_flow*timestep
    out[:,:,i] = col   
        
    if verbose: print "Calculation took %0.2fs"%(time.time()-start)
    return out, iterations

In [4]:
out, iterations = run_column(cap_start=10**-4.8, anal_start=10**-8.0, protein_start=10**-10.0, \
                             kcap = 10**7.7, kanal = 10**9.3, verbose=True, \
                            dtype=longdouble)

Column volume is 52.50 ul
Plate volume is 0.53 ul
6299
pf timestep product: 0.031746031746
Calculation took 1.26s


/Users/richardkwant/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipython-3.2.0-py2.7.egg/IPython/kernel/__main__.py:16: RuntimeWarning: invalid value encountered in true_divide


In [8]:
#animation of the setup
t = 0

cap = out[:,cap_i,t]
prot_cap = out[:,prot_cap_i,t]
prot = out[:,prot_i,t] 
anal = out[:,anal_i,t] 
prot_anal = out[:,prot_anal_i,t] 
tprot = prot+prot_cap+prot_anal
tanal = anal+prot_anal

fig = figure(1, figsize=(10, 5))
clf()

scaling = tprot.sum() / anal[0]

#plot(cap, 'o-', label='cap')
subplot(211)
prot_cap_line, = plot(prot_cap, '-', label='prot-cap')
prot_line, = plot(prot, '-', label='prot')
anal_line, = plot(anal*scaling, '-', label='anal')
prot_anal_line, = plot(prot_anal, label='prot_anal')
legend(loc='upper left')
ylim((0, tprot.sum()*1.2))
grid()
xticks([13, 31, 48, 58], ['analyte_stop', 'protein', 'capture', 'stop'])

subplot(212)
together_line, = plot(prot + prot_cap+prot_anal, '-', label='prot')

ylim((0, tprot.sum()*1.2))
grid()

xticks([13, 31, 48, 58], ['analyte_stop', 'protein', 'capture', 'stop'])

if True:
    # Set up formatting for the movie files
    #Writer = animation.writers['ffmpeg']
    #writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    
    def animate(i):
        i=int(i)
        prot_cap_line.set_ydata(out[:,prot_cap_i,i])
        prot_line.set_ydata(out[:,prot_i,i])
        anal_line.set_ydata(out[:,anal_i,i]*scaling)
        prot_anal_line.set_ydata(out[:,prot_anal_i,i])
        together_line.set_ydata(out[:,prot_i,i] + out[:,prot_anal_i,i] + out[:,prot_cap_i,i])

    ani = animation.FuncAnimation(fig, animate, linspace(0, iterations-1, 200),
        interval=10, blit=False, repeat=False)
    #ani.save('im.mp4', writer=writer)
    
savefig('animation\\%d-b.pdf'%t)
    

figure(2)
clf()
plot(prot_anal / (prot*anal))

/Users/richardkwant/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipython-3.2.0-py2.7.egg/IPython/kernel/__main__.py:58: RuntimeWarning: invalid value encountered in true_divide


#heatmaps

In [44]:
#2D heatmap
#analyte concentration
xstart, xstop, xdim = -11, -4.5, 20
#capture concentration
ystart, ystop, ydim = -9, -4.0, 20
pconc = -10.0
kcapture, kanalyte = 1.0/(20*10**-9.0), 1.0/(0.5*10**-9.0)
out = zeros((ydim, xdim))

xs, ys = linspace(xstart, xstop, xdim), linspace(ystart, ystop, ydim)

f=FloatProgress(min = 0, max = xdim)
display(f)
for i,analpow in enumerate(xs):
    f.value=i+1
    for j,cappow in enumerate(ys):
        
        result, iterations = run_column(anal_start = 10**analpow, cap_start = 10**cappow, protein_start = 10**pconc, \
                                        kcap=kcapture, kanal=kanalyte, timestep=1.0, stop=2.0)
        t = -1
        prot_cap = result[:,prot_cap_i, t]
        prot = result[:,prot_i, t] 
        prot_anal = result[:,prot_anal_i, t]
        
        allprot = prot_cap+prot+prot_anal
        
        first_spot, second_spot = abs(allprot[48]), abs(allprot[58])
        #print first_spot, second_spot
        out[j,i] = log10(second_spot/first_spot)

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:16: RuntimeWarning: invalid value encountered in true_divide


In [45]:
figure(3)
clf()

vrange = 2

ypad, xpad = 0.5*((ystop-ystart)/(ydim-1)), 0.5*((xstop-xstart)/(xdim-1))
ymin, ymax, xmin, xmax = ystart-ypad, ystop + ypad, xstart-xpad, xstop+xpad

imshow(out, interpolation='spline16', cmap='bwr', vmin=-vrange, vmax=vrange, \
       extent=(xmin, xmax, ymax, ymin),\
      aspect=(xstop-xstart)/(ystop-ystart))

xticks(xs, ["%0.1f"%i for i in xs], rotation='vertical')
yticks(ys, ["%0.1f"%i for i in ys])
colorbar()

xlabel('analyte concentration')
ylabel('capture concentration')
title('log([P])=%0.2f, log(Kcap)=%0.2f, log(Kanal)=%0.2f'%(pconc, log10(kcapture), log10(kanalyte)))

vlines(-9, ymin, ymax)

tight_layout()
draw()

savefig('heatmaps\\new-analytical-p_%0.2f-kc_%0.2f-ka_%0.2f.pdf'%(pconc, log10(kcapture), log10(kanalyte)))

#Heatmaps 2

In [158]:
#2D heatmap
#analyte concentration
xstart, xstop, xdim = 4, 10, 10
#capture concentration
ystart, ystop, ydim = -9, -4.0, 10
pconc = -10.0
kcapture, kanalyte = 1.0/(0.5*10**-9.0), 1.0/(20*10**-9.0)
analyte_concentration = 0*10**-8.0
out = zeros((ydim, xdim))

xs, ys = linspace(xstart, xstop, xdim), linspace(ystart, ystop, ydim)

f=FloatProgress(min = 0, max = xdim)
display(f)
for i,kcappow in enumerate(xs):
    f.value=i+1
    for j,cappow in enumerate(ys):
        
        result, iterations = run_column(anal_start = analyte_concentration, cap_start = 10**cappow, protein_start = 10**pconc, \
                                        kcap=10**kcappow, kanal=kanalyte, timestep=1.0, stop=2.0)
        t = -1
        prot_cap = result[:,prot_cap_i, t]
        prot = result[:,prot_i, t] 
        prot_anal = result[:,prot_anal_i, t]
        
        allprot = prot_cap+prot+prot_anal
        
        first_spot, second_spot = abs(allprot[48]), abs(allprot[58])
        #print first_spot, second_spot
        out[j,i] = log10(second_spot/first_spot)

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:16: RuntimeWarning: invalid value encountered in true_divide


In [159]:
figure(3)
clf()

vrange = 2

ypad, xpad = 0.5*((ystop-ystart)/(ydim-1)), 0.5*((xstop-xstart)/(xdim-1))
ymin, ymax, xmin, xmax = ystart-ypad, ystop + ypad, xstart-xpad, xstop+xpad

imshow(out, interpolation='none', cmap='bwr', vmin=-vrange, vmax=vrange, \
       extent=(xmin, xmax, ymax, ymin),\
      aspect=(xstop-xstart)/(ystop-ystart))

xticks(xs, ["%0.1f"%i for i in xs], rotation='vertical')
yticks(ys, ["%0.1f"%i for i in ys])
colorbar()

xlabel('capture ka')
ylabel('capture concentration')
title('log([P])=%0.2f, log([analyte])=%0.2f, log(Kanal)=%0.2f'%(pconc, log10(analyte_concentration), log10(kanalyte)))

#vlines(-9, ymin, ymax)

tight_layout()
draw()

savefig('heatmaps\\analytical-p_%0.2f-an_conc_%0.2f-ka_%0.2f.pdf'%(pconc, log10(analyte_concentration), log10(kanalyte)))

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:17: RuntimeWarning: divide by zero encountered in log10
C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:24: RuntimeWarning: divide by zero encountered in log10
